In [1]:
from pyparsing import \
        Literal, Word, ZeroOrMore, OneOrMore, Group, Dict, Optional, \
        printables, ParseException, restOfLine, empty, \
        Combine, nums, alphanums, Suppress, SkipTo, Forward, printables, alphas
import pprint

In [15]:
#ssn ::= num+ '-' num+ '-' num+
#num ::= '0' | '1' | '2' etc
dash = '-'
ssn = Combine(Word(nums, exact=3) +
                 dash + Word(nums, exact=2) +
                 Suppress('-') + Word(nums, exact=4))
target = '123-45-6789'
result = ssn.parseString(target)
print(result)

['123-456789']


In [ ]:
class PacKiller:
    def __init__(self, package, environments, standalones):
        '''
       *package* is the package name: ex "xy" 
       *environments* is a list of the environments provided by the package:
           [ "xyenvirons", ]
        *standalones* is a list of macros that the package also provides:
           [ "xymatrix" ]
        '''
        pass

In [19]:
example_ini = '''[DEFAULT]
ServerAliveInterval = 45
Compression = yes
CompressionLevel = 9
ForwardX11 = yes

[bitbucket.org]
User = hg

[topsecret.server.com]
Port = 50022
ForwardX11 = no'''

In [24]:
inibnf = None
def inifile_BNF():
    global inibnf
    
    if not inibnf:

        # punctuation
        lbrack = Literal("[").suppress()
        rbrack = Literal("]").suppress()
        equals = Literal("=").suppress()
        semi   = Literal(";")
        
        comment = semi + Optional( restOfLine )
        
        nonrbrack = "".join( [ c for c in printables if c != "]" ] ) + " \t"
        nonequals = "".join( [ c for c in printables if c != "=" ] ) + " \t"
        
        sectionDef = lbrack + Word( nonrbrack ) + rbrack
        keyDef = ~lbrack + Word( nonequals ) + equals + empty + restOfLine
        # strip any leading or trailing blanks from key
        def stripKey(tokens):
            tokens[0] = tokens[0].strip()
        keyDef.setParseAction(stripKey)
        
        # using Dict will allow retrieval of named data fields as attributes of the parsed results
        inibnf = Dict( ZeroOrMore( Group( sectionDef + Dict( ZeroOrMore( Group( keyDef ) ) ) ) ) )
        
        inibnf.ignore( comment )
        
    return inibnf


pp = pprint.PrettyPrinter(2)

def test( strng ):
    print(strng)
    try:
        iniFile = open(strng)
        iniData = "".join( iniFile.readlines() )
        bnf = inifile_BNF()
        tokens = bnf.parseString( iniData )
        pp.pprint( tokens.asList() )

    except ParseException as err:
        print(err.line)
        print(" "*(err.column-1) + "^")
        print(err)
    
    iniFile.close()
    print()
    return tokens
test('../../example.ini')

../../example.ini
[ [ 'DEFAULT',
    ['ServerAliveInterval', '45'],
    ['Compression', 'yes'],
    ['CompressionLevel', '9'],
    ['ForwardX11', 'yes']],
  ['bitbucket.org', ['User', 'hg']],
  ['topsecret.server.com', ['Port', '50022'], ['ForwardX11', 'no']]]



([(['DEFAULT', (['ServerAliveInterval', '45'], {}), (['Compression', 'yes'], {}), (['CompressionLevel', '9'], {}), (['ForwardX11', 'yes'], {})], {'ServerAliveInterval': ['45'], 'Compression': ['yes'], 'CompressionLevel': ['9'], 'ForwardX11': ['yes']}), (['bitbucket.org', (['User', 'hg'], {})], {'User': ['hg']}), (['topsecret.server.com', (['Port', '50022'], {}), (['ForwardX11', 'no'], {})], {'Port': ['50022'], 'ForwardX11': ['no']})], {'DEFAULT': [([(['ServerAliveInterval', '45'], {}), (['Compression', 'yes'], {}), (['CompressionLevel', '9'], {}), (['ForwardX11', 'yes'], {})], {'ServerAliveInterval': ['45'], 'Compression': ['yes'], 'CompressionLevel': ['9'], 'ForwardX11': ['yes']})], 'bitbucket.org': [([(['User', 'hg'], {})], {'User': ['hg']})], 'topsecret.server.com': [([(['Port', '50022'], {}), (['ForwardX11', 'no'], {})], {'Port': ['50022'], 'ForwardX11': ['no']})]})

In [77]:
alphaword = Word(alphas)
integer = Word(nums)
sexp = Forward()
LPAREN = Suppress("(")
RPAREN = Suppress(")")
sexp << OneOrMore( alphaword | integer | ( LPAREN + ZeroOrMore(sexp) + RPAREN ))
tests = """\
 red
 100 ( hi )
 ( red 100 blue )
 ( green ( ( 1 2 ) mauve ) plaid () )""".splitlines()
for t in tests:
    print(t)
    print(sexp.parseString(t))
    print()

 red
['red']

 100 ( hi )
['100', 'hi']

 ( red 100 blue )
['red', '100', 'blue']

 ( green ( ( 1 2 ) mauve ) plaid () )
['green', '1', '2', 'mauve', 'plaid']



In [2]:
with open('../tests/tex_files/reinhardt/reinhardt-optimal-control.tex', 'r') as rein_file:
    rein = rein_file.read()

In [8]:
cstikzfig = Literal("\\tikzfig").suppress()
lbrace = Literal('{').suppress()
rbrace = Literal('}').suppress()
parens = Word("()%\\")
inside = SkipTo(rbrace)
allchars = Word(printables, excludeChars="{}")
inside = ZeroOrMore(allchars)
inside.setParseAction(lambda tok: " ".join(tok))
content = Forward()
content << OneOrMore(allchars|(lbrace + ZeroOrMore(content) + rbrace))
#content << (allchars + lbrace + ZeroOrMore(content) + rbrace)
content.setParseAction(lambda tok: " ".join(tok))
tikzfig = cstikzfig + lbrace + inside + rbrace + lbrace + inside + rbrace + lbrace + content + rbrace

search_res = tikzfig.searchString(rein)

for k,r in enumerate(search_res):
    name, expl, text  = r
    print(k,' ', name,' -- ', expl[:15],' -- ', text[:25], '...', text[-25:])
    #name, expl = r
    #print(k, ' ',name,' -- ', expl[:15],'...',expl[-15:])

0   octagon  --  A smoothed octa  --  % data generate in Mathem ... ., -0.229378) -- (1., 0);
1   dubins  --  This research w  --  \begin scope \draw (-1,0) ... + \x*\x))/4 ); \end scope
2   hyperboloid  --  The cost is the  --  \begin scope \draw (0,-1) ... p(-\x))/2.0 ); \end scope
3   ideal  --  The star inequa  --  \def\rt 1.732 \begin scop ... rcle (\rt cm); \end scope
4   star  --  After an affine  --  \def\rt 1.732 \def\R 2 \b ...  circle (1cm); \end scope
5   traj-upper  --  The trajectorie  --  \def\rt 1.732 % e3 \begin ... m); \end scope \end scope
6   tri  --  The trajectory   --  \def\rt 1.732 \begin scop ... smalldot 0,0 ; \end scope
7   cost  --  The graph inter  --  \begin scope [xscale=0.5, ... ,black] $\y$ ; \end scope


In [31]:
print(rein[:10000])

% Thomas Hales
% Nov 8, 2016
% arXiv version, Mar 3, 2017.
% Reinhardt Conjecture 2, control theory
%[12pt]

% hexameral -> hexagonally symmetric
% domain -> disk

% http://english.stackexchange.com/questions/79847/parametrise-or-parameterise-a-curve
% parameter spell, using parametrization.


\documentclass{article}

\usepackage{graphicx}
\usepackage{amsthm,xcolor}
\usepackage{amsfonts}
\usepackage{amsmath}
\usepackage{amscd}
\usepackage{amssymb}
\usepackage{alltt}
\usepackage{url}
\usepackage{ellipsis}
% 
%tikz graphics
%\usepackage{xcolor} % to remove color.
\usepackage{tikz} % 
\usetikzlibrary{chains,shapes,arrows,%
 trees,matrix,positioning,decorations,fadings}
%\usepackage[framemethod=TikZ]{mdframed}

\def\tikzfig#1#2#3{%
\begin{figure}[htb]%
  \centering
\begin{tikzpicture}#3
\end{tikzpicture}
  \caption{#2}
  \label{fig:#1}%
\end{figure}%
}
\def\smalldot#1{\draw[fill=black] (#1) %
 node [inner sep=1.3pt,shape=circle,fill=black] {}}
\newtheorem{theorem}{Theorem}[subsection]
\new